In [ ]:
#!pip install efficientnet_pytorch

In [25]:
!pip install efficientnet_pytorch

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-cp37-none-any.whl size=16443 sha256=6fed912e9e2da24b690c6d95446041653c194820936f51400e48e324f8bcda23
  Stored in directory: /root/.cache/pip/wheels/84/27/aa/c46d23c4e8cc72d41283862b1437e0b3ad318417e8ed7d5921
Successfully built efficientnet-pytorch


In [26]:
from efficientnet_pytorch import EfficientNet

In [1]:
import os
import multiprocessing
import numpy as np
import cv2
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms

In [2]:
if torch.cuda.is_available():  
  dev = "cuda:0" 
else:  
  dev = "cpu"  

device = torch.device(dev)
print(device)

cuda:0


In [3]:
!nvidia-smi

Wed May  5 15:47:32 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    32W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
x = np.load('/content/drive/MyDrive/Cache/x_test.npy')
y = torch.load('/content/drive/MyDrive/Cache/test_y.pt')

In [5]:
class Date_3d_Cached(torch.utils.data.Dataset):
    def __init__(self, x, y):
      self.x = torch.from_numpy(x)
      self.y = y
    
    def __len__(self):
      return self.x.shape[0]

    def __getitem__(self, idx):
      return self.x[idx] /255.0, self.y[idx]

In [9]:
x_6 = x[:, :6, ...]

In [10]:
data = Date_3d_Cached(x_6, y)

In [11]:
#https://towardsdatascience.com/implementing-the-new-state-of-the-art-mish-activation-with-2-lines-of-code-in-pytorch-e7ef438a5ee7
def mish(x): 
  return (x * torch.tanh(nn.functional.softplus(x)))



In [14]:
#old model loading
# cp = torch.load("/content/drive/MyDrive/Models_R50_reg_dropout/ep_126_loss_37.18821668624878_acc_tensor(97.5000, device='cuda:0')val_loss_best.pt") 
# model = Model_3d(1024, len(views), 10)
# model.load_state_dict(cp['state_dict'])

In [17]:
views = ['bottom', 'side_1', 'side_2', 'side_3', 'side_4', 'top'] #, 'top_1', 'top_2', 'top_3', 'top_4', 'bot_1', 'bot_2', 'bot_3', 'bot_4']
testing_data = Date_3d_Cached(x, y)

In [15]:
test_loader = torch.utils.data.DataLoader(data, batch_size= 8, num_workers=os.cpu_count())
if torch.cuda.is_available():  
  dev = "cuda:0" 
else:  
  dev = "cpu"  

device = torch.device(dev)
print(device) 

cuda:0


In [16]:
!nvidia-smi

Wed May  5 15:53:56 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0    39W / 250W |   1059MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [17]:
print(device)

cuda:0


In [58]:
#https://machinelearningmastery.com/pytorch-tutorial-develop-deep-learning-models/
class Model_3d(nn.Module):
    '''
    A model to use on 3d objects
    '''
    def __init__(self, fc_nodes, no_views, no_classes):
      super(Model_3d, self).__init__()
      self.resnet = torchvision.models.resnet50(pretrained=True) #https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

      num_features = self.resnet.fc.out_features

      self.fcs = nn.ModuleList()

      for idx in range(no_views):
        self.fcs.append(nn.Linear(num_features, fc_nodes))

      self.final_dense = nn.Linear(fc_nodes * no_views, no_classes)
    
    def forward(self, x):
      features = []

      for i, fc in enumerate(self.fcs):
        sample = x[:, i, ...]
        r_out = self.resnet(sample)

        fc_out = fc(r_out)
        features.append(fc_out)
      
      stack = torch.stack(features, 1)

      dense_out = self.final_dense(stack.view([x.shape[0], -1]))

      return torch.nn.functional.softmax(dense_out)

In [59]:
model = Model_3d(1024, 6, 10)
model.to(device)
print(device)

cuda:0


In [68]:
test_dir = '/content/drive/MyDrive/Models_R50_mish'

In [71]:
print(len(data))
criterion = nn.CrossEntropyLoss()

model_dict_for_map = {}

map_cache = {'y':[], 'y_hat':[]}

models = os.listdir(test_dir)
#models.remove('logs.csv')
models.remove('last.pt')

old_load = False

epochs_done = []

best_acc = 0

for model_path in models:
  if model_path[-4:] == '.csv':
    continue
  try:
    epoch = model_path.split('_')[1]
  except:
    continue


  if epoch in epochs_done:
    continue
  else:
    epochs_done.append(epoch)

  print(model_path)
  running_loss = 0.0
  correct = 0
  total = 0

  if dev == 'cpu':
    model = torch.load(os.path.join(test_dir, model_path), map_location=torch.device('cpu'))
  else:
    model = torch.load(os.path.join(test_dir, model_path))
  
  if model_path == 'last.pt' or old_load:
    cached = model['state_dict']
    model = Model_3d(1024, 6, 10)
    model.load_state_dict(cached)

  model.to(device)
  model.eval()
  with torch.no_grad():
      for i, data in enumerate(test_loader):
          
          # get the inputs; data is a list of [inputs, labels]
          inputs, labels = data
          inputs = inputs.to(device)
          labels = labels.to(device)            


          # forward + backward + optimize
          outputs = model(inputs)
          #loss = criterion(outputs, labels)

          map_cache['y'].append(labels)
          map_cache['y_hat'].append(outputs)

          correct += (outputs.argmax(1) == labels).float().sum() #https://stackoverflow.com/questions/51503851/calculate-the-accuracy-every-epoch-in-pytorch
          total += labels.shape[0]


  accuracy = 100 * correct / total
  print(" Test Accuracy = {}".format(accuracy))
  if accuracy > best_acc:
    best_acc = accuracy
    best_model = model_path
    best_map_cach = map_cache
  map_cache = {'y':[], 'y_hat':[]}

2
ep_79_loss_37.28839683532715_acc_tensor(97.5000, device='cuda:0')acc_best.pt


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


 Test Accuracy = 92.40087890625
ep_92_loss_37.20040142536163_acc_tensor(97.7500, device='cuda:0')val_loss_best.pt
 Test Accuracy = 91.62995147705078


In [72]:
map_cache = best_map_cach

In [73]:
import os

string = '''
Algorithm	ModelNet40
Classification
(Accuracy)	ModelNet40
Retrieval
(mAP)	ModelNet10
Classification
(Accuracy)	ModelNet10
Retrieval
(mAP)
RS-CNN[63]	93.6%	-	-	-
LP-3DCNN[62]	92.1%	-	94.4%	-
LDGCNN[61]	92.9%	-	-	-
Primitive-GAN[60]	86.4%	-	92.2%	-
3DCapsule [59]	92.7%	-	94.7%	-
3D2SeqViews [58]	93.40%	90.76%	94.71%	92.12%
OrthographicNet [57]	-	-	88.56%	86.85%
Ma et al. [56]	91.05% 	84.34% 	95.29% 	93.19%
MLVCNN [55]	94.16%	92.84%	- 	-
iMHL [54]	97.16%	- 	- 	-
HGNN [53]	96.6%	- 	- 	-
SPNet [52]	92.63% 	85.21% 	97.25% 	94.20%
MHBN [51] 	94.7	-	95.0	-
VIPGAN [50]	91.98	89.23	94.05	90.69
Point2Sequence [49]	92.60	-	95.30	-
Triplet-Center Loss [48]	-	88.0%	-	-
PVNet[47]	93.2%	89.5%	- 	-
GVCNN[46]	93.1%	85.7%	- 	-
MLH-MV[45]	93.11%		94.80%	
MVCNN-New[44]	95.0%			
SeqViews2SeqLabels[43]	93.40%	89.09% 	94.82%	91.43%
G3DNet[42]	91.13%		93.1%	
VSL [41]	84.5%		91.0%	
3D-CapsNets[40]	82.73%	70.1% 	93.08%	88.44%
KCNet[39]	91.0% 		94.4% 	
FoldingNet[38]	88.4% 		94.4% 	
binVoxNetPlus[37]	85.47% 		92.32%	
DeepSets[36]	90.3%			
3D-DescriptorNet[35]			92.4%	
SO-Net[34]	93.4%		95.7%	
Minto et al.[33]	89.3%		93.6%	
RotationNet[32]	97.37%		98.46%	
LonchaNet[31]			94.37 	
Achlioptas et al. [30]	84.5%		95.4% 	
PANORAMA-ENN [29]	95.56%	86.34% 	96.85% 	93.28%
3D-A-Nets [28]	90.5% 	80.1% 		
Soltani et al. [27]	82.10%			
Arvind et al. [26]	86.50%			
LonchaNet [25]			94.37% 	
3DmFV-Net [24]	91.6% 		95.2%	
Zanuttigh and Minto [23]	87.8% 		91.5%	
Wang et al. [22]	93.8%			
ECC [21]	83.2%		90.0%	
PANORAMA-NN [20]	90.7%	83.5%	91.1%	87.4%
MVCNN-MultiRes [19]	91.4%			
FPNN [18]	88.4% 			
PointNet[17]	89.2%			
Klokov and Lempitsky[16]	91.8% 		94.0%	
LightNet[15]	88.93%		93.94%	
Xu and Todorovic[14]	81.26% 		88.00%	
Geometry Image [13]	83.9% 	51.3%	88.4%	74.9%
Set-convolution [11]	90%			
PointNet [12]			77.6%	
3D-GAN [10]	83.3%		91.0%	
VRN Ensemble [9]	95.54%		97.14%	
ORION [8]			93.8%	
FusionNet [7]	90.8%		93.11%	
Pairwise [6]	90.7%		92.8%	
MVCNN [3]	90.1%	79.5%		
GIFT [5]	83.10%	81.94% 	92.35%	91.12%
VoxNet [2]	83%		92%	
DeepPano [4]	77.63%	76.81%	85.45%	84.18%
3DShapeNets [1]	77%	49.2%	83.5%	68.3%
'''

modelNet10acc = []
modelNet10map = []

lines = string.split('\n')
for line in lines:
  tabed = line.split('\t')
  if len(tabed) > 3:
    val_acc = tabed[3]
    if len(val_acc) > 3:
      rem = val_acc.split('%')[0]
      modelNet10acc.append(float(rem))
    map = tabed[4]
    if len(map) > 3:
      rem = map.split('%')[0]
      modelNet10map.append(float(rem))

modelNet10acc.sort(reverse=True)
modelNet10map.sort(reverse=True)

print(modelNet10acc)
print(modelNet10map)

[98.46, 97.25, 97.14, 96.85, 95.7, 95.4, 95.3, 95.29, 95.2, 95.0, 94.82, 94.8, 94.71, 94.7, 94.4, 94.4, 94.4, 94.37, 94.37, 94.05, 94.0, 93.94, 93.8, 93.6, 93.11, 93.1, 93.08, 92.8, 92.4, 92.35, 92.32, 92.2, 91.5, 91.1, 91.0, 91.0, 90.0, 88.56, 88.4, 88.0, 85.45, 83.5, 77.6]
[94.2, 93.28, 93.19, 92.12, 91.43, 91.12, 90.69, 88.44, 87.4, 86.85, 84.18, 74.9, 68.3]


In [74]:
for idx, val in enumerate(modelNet10acc):
  if best_acc > val:
    print('place', idx + 1)
    break
print(best_model)

place 29
ep_79_loss_37.28839683532715_acc_tensor(97.5000, device='cuda:0')acc_best.pt


In [75]:

per_class_probs = [[] for _ in range(10)]
per_class_truth = [[] for _ in range(10)]

confusion_labels = []
confusion_pred = []


for idx in range(len(map_cache['y'])):
  this_batch_label = map_cache['y'][idx]
  this_batch_pred = map_cache['y_hat'][idx]

  for item_idx in range(this_batch_label.shape[0]):
    this_item_label = this_batch_label[item_idx].item()
    this_item_pred = this_batch_pred[item_idx].cpu().numpy()

    confusion_labels.append(this_item_label)
    confusion_pred.append(this_item_pred.argmax())

    for cls_idx in range(this_item_pred.shape[0]):
      if cls_idx == this_item_label:
        per_class_truth[cls_idx].append(1.0)
        per_class_probs[cls_idx].append(this_item_pred[cls_idx].item())
      else:
        per_class_truth[cls_idx].append(0.0)
        per_class_probs[cls_idx].append(this_item_pred[cls_idx].item())



In [76]:
from sklearn.metrics import average_precision_score

per_class_map = []

for idx in range(len(per_class_probs)):
  this_probs = np.array(per_class_probs[idx])
  this_true = np.array(per_class_truth[idx])
  this_map = average_precision_score(this_true, this_probs)
  per_class_map.append(this_map.item())

map = (sum(per_class_map)/len(per_class_map)) * 100
print('map', float(map))

map 95.81366568498761


In [57]:
for idx, this_map in enumerate(modelNet10map):
  if map > this_map:
    print('place', idx + 1)
    break
print(best_model)

place 1
Copy of ep_126_loss_37.18821668624878_acc_tensor(97.5000, device='cuda:0')val_loss_best.pt


In [56]:
print(modelNet10map)

[94.2, 93.28, 93.19, 92.12, 91.43, 91.12, 90.69, 88.44, 87.4, 86.85, 84.18, 74.9, 68.3]


In [47]:
from sklearn.metrics import confusion_matrix
#y_true = [2, 0, 2, 2, 0, 1]
#y_pred = [0, 0, 2, 2, 0, 2]
confusion_matrix(confusion_labels, confusion_pred)

array([[ 79,   0,   0,   7,   0,   0,   0,   0,   0,   0],
       [  1,  80,   3,   1,   0,   0,   1,   0,   0,   0],
       [  0,  21,  77,   2,   0,   0,   0,   0,   0,   0],
       [ 15,   0,   1,  70,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0, 100,   0,   0,   0,   0,   0],
       [  0,   0,   1,   1,   0,  98,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0, 100,   0,   0,   0],
       [  0,   0,   0,   1,   0,   0,   0,  98,   0,   1],
       [  0,   0,   0,   0,   0,   0,   0,   0, 100,   0],
       [  0,   0,   0,   2,   0,   0,   0,   0,   0,  48]])

In [ ]:
root_dir = '/content/drive/MyDrive/Models_exp2.14.2'

if torch.cuda.is_available():  
  dev = "cuda:0" 
else:  
  dev = "cpu"  

device = torch.device(dev)
print(device)

#https://machinelearningmastery.com/pytorch-tutorial-develop-deep-learning-models/
class Model_Encoder(nn.Module):
    '''
    A model to use on 3d objects
    '''
    def __init__(self):
      super(Model_Encoder, self).__init__()
      self.inception = torchvision.models.inception_v3(pretrained=False, aux_logits=False) #https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
      
    def forward(self, x):
      
      x =  nn.functional.interpolate(x, 299)

      x = self.inception(x)


      return x

#https://analyticsindiamag.com/how-to-implement-convolutional-autoencoder-in-pytorch-with-cuda/
class Model_Decoder(nn.Module):
    '''
    A model to use on 3d objects
    '''
    def __init__(self):
      super(Model_Decoder, self).__init__()
      self.deconv1 = nn.ConvTranspose2d(40, 32, 2, stride=2)
      self.deconv2 = nn.ConvTranspose2d(32, 24, 2, stride=2)
      self.deconv3 = nn.ConvTranspose2d(24, 20, 2, stride=2)
      self.deconv4 = nn.ConvTranspose2d(20, 16, 2, stride=2)
      self.deconv5 = nn.ConvTranspose2d(16, 8, 2, stride=2)
      self.deconv6 = nn.ConvTranspose2d(8, 3, 2, stride=2)
      
      
    def forward(self, x):
      x = x.view([-1, 40, 5, 5])
      x = self.deconv1(x) #10 x 10
      x = self.deconv2(x) #20 x 20
      x = self.deconv3(x) #40 x 40
      x = self.deconv4(x) #80 x 80
      x = self.deconv5(x) #160 x 160
      x = self.deconv6(x) #320 x 320

      x = nn.functional.interpolate(x, 224)

      return nn.functional.sigmoid(x)

class Date_3d_Cached(torch.utils.data.Dataset):
    def __init__(self, x):
      self.x = torch.from_numpy(x).view([-1, 3, 224, 224])
    
    def __len__(self):
      return self.x.shape[0]

    def __getitem__(self, idx):
      x = self.x[idx] /255.0
      return x

views = ['bottom', 'side_1', 'side_2', 'side_3', 'side_4', 'top', 'top_1', 'top_2', 'top_3', 'top_4', 'bot_1', 'bot_2', 'bot_3', 'bot_4']
x = np.load('/content/drive/MyDrive/Cache/x_compressed.npz')['arr_0']

training_data = Date_3d_Cached(x)
#90/10 train val split
train_len = int(len(training_data) * .9)
val_len = len(training_data) - train_len

batch_size = 96

encoder = Model_Encoder()
encoder.to(device)

training_loader = torch.utils.data.DataLoader(training_data, batch_size= batch_size, shuffle=True, num_workers=os.cpu_count())

#https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html
criterion = nn.MSELoss()
optimizer_encoder = optim.SGD(encoder.parameters(), lr=0.001, momentum=0.9)

decoder = Model_Decoder()
decoder.to(device)
optimizer_decoder = optim.SGD(decoder.parameters(), lr=0.001, momentum=0.9)

#https://medium.com/analytics-vidhya/saving-and-loading-your-model-to-resume-training-in-pytorch-cb687352fa61
def save_ckp(state, model_path):
    torch.save(state, model_path)


if not os.path.exists(root_dir):
    os.makedirs(root_dir)


low_val_loss = 30000000
best_acc = -1
epoch = 0

val_loss_path = None
acc_path = None
#https://stackoverflow.com/questions/8078330/csv-writing-within-loop
import csv
with open(os.path.join(root_dir, 'logs.csv'), 'w') as file_csv:
  writer=csv.writer(file_csv, delimiter=',',lineterminator='\n',)
  writer.writerow(['epoch', 'loss'])


  for epoch in range(1, 101):
      row = [epoch]
      is_best = False
      running_loss = 0.0
      correct = 0
      total = 0

      encoder.train()
      decoder.train()

      t0 = time.time()
      for i, data in enumerate(training_loader):
          inputs = data


          inputs = inputs.to(device)

          # zero the parameter gradients
          optimizer_encoder.zero_grad()
          optimizer_decoder.zero_grad()

          # forward + backward + optimize
          encoded = encoder(inputs)
          decoded = decoder(encoded)

          loss = criterion(decoded, inputs)
          loss.backward()

          optimizer_encoder.step()
          optimizer_decoder.step()

          # print statistics
          running_loss += loss.item()

      print('epoch', epoch)
      
      print('{} seconds'.format(time.time() - t0))
      print('loss', running_loss)
      row.append(running_loss)

      model_name = 'ep_' + str(epoch) + '_loss_' + str(running_loss) + '.pt'
      full_model_path = os.path.join(root_dir, model_name)
      checkpoint = {
          'epoch': epoch + 1,
          'state_dict': encoder.state_dict(),
          'optimizer': optimizer_encoder.state_dict()
      }
      if running_loss < low_val_loss:
        if val_loss_path and os.path.exists(val_loss_path):
          open(val_loss_path, 'w').close() #overwrite and make the file blank instead - ref: https://stackoverflow.com/a/4914288/3553367
          os.remove(val_loss_path)
        val_loss_path = full_model_path.replace('.pt', 'val_loss_best.pt')
        save_ckp(encoder, val_loss_path)
        low_val_loss = running_loss
        print('new low loss')

      full_model_path = os.path.join(root_dir, 'last.pt')
      save_ckp(checkpoint, full_model_path)
          

      
      running_loss = 0.0
      writer.writerow(row)
  print('Finished Training')

cuda:0


/usr/local/lib/python3.7/dist-packages/torchvision/models/inception.py:82: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  ' due to scipy/scipy#11299), please set init_weights=True.', FutureWarning)
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


epoch 1
239.7427396774292 seconds
loss 126.00858446955681
new low loss
epoch 2
239.72910857200623 seconds
loss 110.50437399744987
new low loss
epoch 3
239.97946310043335 seconds
loss 101.86395159363747
new low loss
epoch 4
240.27999258041382 seconds
loss 97.12703458964825
new low loss
epoch 5
241.04839301109314 seconds
loss 94.42442092299461
new low loss
epoch 6
240.6546585559845 seconds
loss 92.8531966805458
new low loss
epoch 7
240.80921959877014 seconds
loss 91.85168365389109
new low loss
epoch 8
240.72932958602905 seconds
loss 91.3075773268938
new low loss
epoch 9
240.74599313735962 seconds
loss 90.98079742491245
new low loss
epoch 10
241.2099769115448 seconds
loss 90.7177165299654
new low loss
epoch 11
241.06901049613953 seconds
loss 90.61321486532688
new low loss
epoch 12
240.8540759086609 seconds
loss 90.54805183410645
new low loss
epoch 13
240.99463319778442 seconds
loss 90.44873517751694
new low loss
epoch 14
240.89445424079895 seconds
loss 90.45691965520382
epoch 15
240.41304